## 테스트 데이터 로드 및 전처리

In [1]:
import torch
from datasets import load_from_disk
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
test_dataset = load_from_disk('test_dataset')

In [3]:
# base_model tokenizer load
model = 'Qwen/Qwen2-7B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model)

테스트 데이터를 불러와서 입력과 레이블 형태로 전처리합니다.

In [4]:
prompt_lst = []
label_lst = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label)

임의로 20번 샘플을 출력합니다.

In [38]:
# 20번 테스트 샘플 출력
print(prompt_lst[20])

<|im_start|>system
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
문서1: “인재 제일의 삼성이라더니 너무 실망입니다.” 김정미 제일모직 상무(43·사진)가 21년 전 삼성 본관 25층 인사팀의 방명록에 쓴 문장이다. 당시 인사 담당자는 삼성 입사 원서를 받으려고 인사팀을 찾은 그를 힐끗 쳐다보고 말했다. “군필이에요?” 얼굴이 붉어졌지만 김 상무는 지지 않고 대꾸했다. “저는 면제예요. 원서 주세요.”하지만 지원서를 받지 못했다. 여대생이라는 이유에서 거부당한 것이다. 불과 20여년 전, 삼성도 그랬다. “방명록에 학점이나 써놓고 가라”는 직원의 말에 김 상무는 화가 나서 “(삼성에) 실망했다”는 글을 남긴 채 그곳을 나왔다.지난달 30일 저녁 서울 서초동 삼성전자 본사 3층 대회의실에서 20~30대 여성 100여명이 김 상무의 얘기에 귀를 기울였다. 삼성이 여성 임직원의 경험을 외부와 공유하기 위해 만든 ‘여기(女氣) 모여라’ 현장이다. 삼성 트위터, 페이스북, 카카오플러스 등 소셜네트워크서비스(SNS)를 통해 일반인 신청을 받아 초청했다. 이번 강연엔 1000명 넘게 신청이 몰려 10 대 1의 경쟁률을 기록했다. 김 상무는 얘기를 이어갔다. 좌절의 

In [37]:
# 임의의 20번 테스트 샘플 레이블 출력
print(label_lst[20])


여대생이라는 이유로 지원서를 받지 못한 인물은 김정미 제일모직 상무입니다. 김정미 상무는 21년 전 삼성 본관 인사팀을 찾아가 입사 원서를 요청했으나, 여대생이라는 이유로 거부당했습니다. 당시 인사 담당자는 "군필이에요?"라고 물었고, 김 상무가 "저는 면제예요. 원서 주세요."라고 대답했지만, 결국 지원서를 받지 못했습니다. [[ref1]]<|im_end|>



## 기본 모델

In [21]:
# pipeline 객체와 LLM의 입력을 넣으면 LLM의 답변을 출력해주는 함수
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [22]:
model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen2-7B-Instruct', device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
eos_token = tokenizer("<|im_end|>", add_special_tokens=False)["input_ids"][0]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

기본 모델에 임의로 20번 샘플을 넣어서 출력합니다. 기본 모델은 출처를 남기지 않습니다.

In [35]:
result = test_inference(pipe, prompt_lst[20])
print(result)

김정미 제일모직 상무가 해당 질문에 대한 내용이 있습니다. 그녀는 21년 전 삼성 본관 25층 인사팀의 방명록에 "여대생이라는 이유에서 거부당한 것"을 언급하였습니다.


기본 모델에 임의로 400~404번 샘플을 출력합니다. 기본 모델은 지시사항에서 출처를 남기라는 지시를 따르지 않습니다.

In [24]:
for prompt, label in zip(prompt_lst[400:405], label_lst[400:405]):
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

    response:
문서4에 따르면, 신탁통치반대운동이 찬반으로 나뉘어지게 한 언론은 '민족일보'입니다. 문서에서는 "민족일보는 1985년에 체결된 한미경제원조협정에 대한 집중적인 비판을 통해 정권을 무너뜨리려는 시도를 했다"고 기록하고 있습니다.
    label:

잘못된 보도로 인해 신탁통치반대운동이 찬반으로 나뉘게 한 언론은 동아일보입니다. 동아일보의 오보가 신탁 통치 반대 운동을 좌우 양측으로 나뉘게 한 주요 원인 중 하나로 기록되어 있습니다 [[ref2]].<|im_end|>

--------------------------------------------------
    response:
인종에 따른 편향을 실험한 나라는 미국입니다. 문서5에서 언급된 TV 뉴스 실험은 미국에서 진행되었습니다. 이 실험은 가상의 뉴스를 시청하게 된 참가자들이 용의자가 백인인지 흑인인지에 따라 범죄를 우려하는 정도가 달라지는 것을 확인하였습니다.
    label:

인종에 따른 편향을 실험한 나라는 미국입니다. 문서5에 따르면, 미국에서 행해진 TV 뉴스 실험에서 참여자들을 두 그룹으로 나누어 가상으로 만들어진 뉴스를 시청하게 하였고, 아동성폭력을 보도하는 같은 뉴스에서 한 그룹은 용의자가 백인으로 제시된 뉴스를, 다른 그룹은 용의자가 흑인으로 제시된 뉴스를 시청하였습니다. 그 결과 흑인이 용의자로 지목된 뉴스를 시청한 그룹이 보다 더 범죄를 우려하는 반응을 보였고 흑인을 일반적으로 범죄와 연결시키는 편향을 보였습니다 [[ref5]].<|im_end|>

--------------------------------------------------
    response:
2009년에 15명의 교사들이 받은 징계의 원인은 그들이 전국교직원노동조합(전교조)의 시국선언에 참여하였기 때문입니다. 이 시국선언은 국가공무원법의 집단행위 금지 조항과 교원노조법의 정치 활동 금지 조항을 어긴 것이라는 이유로 교육과학기술부로부터 징계를 받았습니다.
    label:

200

## 튜닝 모델

In [9]:
peft_model_id = "qwen2-7b-rag-ko/checkpoint-177"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
fine_tuned_pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)
eos_token = tokenizer("<|im_end|>", add_special_tokens=False)["input_ids"][0]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

튜닝 모델에 임의로 20번 샘플을 넣어서 출력합니다. 튜닝 모델은 출처를 남깁니다.

In [36]:
result = test_inference(fine_tuned_pipe, prompt_lst[20])
print(result)

지원서를 받지 못한 인물의 이름은 김정미 제일모직 상무입니다. [[ref1]]


튜닝 모델에 임의로 400~404번 샘플을 출력합니다. 튜닝 모델은 튜닝이 되었으므로 프롬프트 지시사항에 따라서 출처를 남기는 모습을 보여줍니다.  
물론, 출처를 남기지 않는 샘플들도 존재할 것입니다. 이는 학습이 덜 된 것이므로 학습 데이터의 비율을 높이고 에포크를 높이시기 바랍니다.

In [17]:
for prompt, label in zip(prompt_lst[400:405], label_lst[400:405]):
    print(f"    response:\n{test_inference(fine_tuned_pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

    response:
잘못된 보도로 인해 신탁통치반대운동이 찬반으로 나뉘어지게 한 언론은 동아일보입니다. 문서2에서 언급된 내용에 따르면, 동아일보의 오보가 없었더라도 빠르게 정착되던 냉전 구도는 좌우 이념 대립을 격화시켰을 것이며, 신탁 통치 반대 운동이 좌우 양측으로 나뉘어 찬반이 격렬하게 진행된 이유로 동아일보의 오보가 언급되었습니다 [[ref2]].
    label:

잘못된 보도로 인해 신탁통치반대운동이 찬반으로 나뉘게 한 언론은 동아일보입니다. 동아일보의 오보가 신탁 통치 반대 운동을 좌우 양측으로 나뉘게 한 주요 원인 중 하나로 기록되어 있습니다 [[ref2]].<|im_end|>

--------------------------------------------------
    response:
인종에 따른 편향을 실험한 나라는 미국입니다. 문서5에서 언급된 TV 뉴스 실험은 미국에서 행해졌습니다. 이 실험은 두 그룹의 참여자들에게 동일한 가상의 뉴스를 시청하게 하였는데, 한 그룹은 용의자가 백인으로, 다른 그룹은 용의자가 흑인으로 제시된 뉴스를 시청하게 하였습니다. 이 실험을 통해 인종에 따른 편향을 연구하였습니다 [[ref5]].
    label:

인종에 따른 편향을 실험한 나라는 미국입니다. 문서5에 따르면, 미국에서 행해진 TV 뉴스 실험에서 참여자들을 두 그룹으로 나누어 가상으로 만들어진 뉴스를 시청하게 하였고, 아동성폭력을 보도하는 같은 뉴스에서 한 그룹은 용의자가 백인으로 제시된 뉴스를, 다른 그룹은 용의자가 흑인으로 제시된 뉴스를 시청하였습니다. 그 결과 흑인이 용의자로 지목된 뉴스를 시청한 그룹이 보다 더 범죄를 우려하는 반응을 보였고 흑인을 일반적으로 범죄와 연결시키는 편향을 보였습니다 [[ref5]].<|im_end|>

--------------------------------------------------
    response:
2009년에 15명의 교사들이 받은 징계의 원인은 "전국교직원노동조합(전교조)의 시

## Merge

In [39]:
!python merge.py \
    --base_model_name_or_path Qwen/Qwen2-7B-Instruct \
    --peft_model_path ./qwen2-7b-rag-ko/checkpoint-177 \
    --output_dir ./output_dir

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading base model: Qwen/Qwen2-7B-Instruct
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:17<00:00,  4.42s/it]
Loading PEFT: ./qwen2-7b-rag-ko/checkpoint-177
Running merge_and_unload
[2025-03-15 16:03:38,654] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Model saved to ./output_dir


In [18]:
# 실습 후에는 상단의 Kernel > Shut Down Kernel을 눌러서 종료합니다.